In [28]:
import numpy as np
import pandas as pd

In [29]:
train_path="/home/saurav-neupane/full_codes/dynamic-ride-pricing/data/raw/new-york-city-taxi-fare-prediction/train.csv"
train_df=pd.read_csv(train_path)

In [30]:
train_df.shape

(55423856, 8)

In [31]:
train_df["pickup_datetime"]=pd.to_datetime(train_df["pickup_datetime"])

In [32]:
train_df["pickup_datetime"].head()

0   2009-06-15 17:26:21+00:00
1   2010-01-05 16:52:16+00:00
2   2011-08-18 00:35:00+00:00
3   2012-04-21 04:30:42+00:00
4   2010-03-09 07:51:00+00:00
Name: pickup_datetime, dtype: datetime64[ns, UTC]

In [33]:
train_df["hour"]=train_df["pickup_datetime"].dt.hour

In [34]:
train_df["day_of_week"]=train_df["pickup_datetime"].dt.day_of_week

In [35]:
train_df["is_weekend"]=train_df["day_of_week"].isin([5,6]).astype(int)

In [36]:
train_df["is_peak_hour"]=train_df["hour"].isin([7,8,9,16,17,18]).astype(int)

In [37]:
def haversine_distance(lat1,lon1,lat2,lon2):
    R=6371
    lat1,lon1,lat2,lon2=map(np.radians,[lat1,lon1,lat2,lon2])
    dlat=lat2-lat1
    dlon=lon2-lon1
    a = (np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2    )
    c = 2 * np.arcsin(np.sqrt(a))

    return R * c

In [38]:
train_df["distance_km"]=haversine_distance(train_df["pickup_latitude"],
                                           train_df["pickup_longitude"],
                                           train_df["dropoff_latitude"],
                                           train_df["dropoff_longitude"]
                        )

In [39]:
train_df["distance_km"].describe()

count    5.542348e+07
mean     1.987936e+01
std      3.714150e+02
min      0.000000e+00
25%      1.212417e+00
50%      2.116555e+00
75%      3.875318e+00
max      1.968880e+04
Name: distance_km, dtype: float64

In [40]:
train_df=train_df[train_df["distance_km"]>0]

In [41]:
train_df=train_df[train_df["distance_km"]<=50]

In [42]:
train_df["distance_km"].describe()

count    5.370680e+07
mean     3.347777e+00
std      3.588883e+00
min      8.374409e-05
25%      1.277243e+00
50%      2.175309e+00
75%      3.938344e+00
max      4.998468e+01
Name: distance_km, dtype: float64

In [43]:
train_df=train_df.drop(columns=["pickup_datetime",
                                "pickup_latitude",
                                "pickup_longitude",
                                "dropoff_latitude",
                                "dropoff_longitude",
                                "key"])

In [44]:
train_df.head()

,fare_amount,passenger_count,hour,day_of_week,is_weekend,is_peak_hour,distance_km
0,4.5,1,17,0,0,1,1.030764
1,16.9,1,16,1,0,1,8.450134
2,5.7,2,0,3,0,0,1.389525
3,7.7,1,4,5,1,0,2.799270
4,5.3,1,7,1,0,1,1.999157


In [45]:
train_df.to_csv("/home/saurav-neupane/full_codes/dynamic-ride-pricing/data/processed/train_features.csv", index=False)
